In [2]:
# coding=utf-8
from keras.models import Sequential, load_model
from keras.layers import Dense
import numpy as np
import random
from collections import deque
import sys                                   # Pro pripojeni knihovny Open AI Gym
sys.path.append('/home/xbucha02/libraries')  # Pripojeni knihovny Open AI Gym
import gym
import numpy as np
from keras.models import save_model

BATCH_SIZE = 64
GAMMA = 0.9
EPSILON = 0.5


class DQN:
    def __init__(self, env):
        self.replay = deque(maxlen=10000)
        self.state_dim = env.observation_space.shape[0]
        self.action_dim = env.action_space.n
        self.epsilon = EPSILON

        self.create_model()

    def create_model(self):
        self.model = Sequential()
        self.model.add(Dense(20, activation='relu', input_shape=(self.state_dim,)))
        self.model.add(Dense(20, activation='relu'))
        self.model.add(Dense(self.action_dim, activation='linear'))
        self.model.summary()
        self.model.compile(optimizer='Adam', loss='mse')

    def perceive(self, state, action, reward, next_state, done):
        self.replay.append((state, action, reward, next_state, done))
        if len(self.replay) > BATCH_SIZE:
            self.train_Q_network()

    def train_Q_network(self):
        minbatch = random.sample(self.replay, BATCH_SIZE)
        state_batch = np.vstack([i[0] for i in minbatch])
        action_batch = [i[1] for i in minbatch]
        reward_batch = [i[2] for i in minbatch]
        next_state_batch = np.vstack([i[3] for i in minbatch])
        y_batch = []
#        print("Statetr: {}, {}" .format(state_batch[1], np.shape(next_state_batch[1])))
        Q_value_batch = self.model.predict(next_state_batch)

        for i in range(BATCH_SIZE):
            if minbatch[i][4]:
                reward = reward_batch[i]
            else:
                reward = reward_batch[i] + GAMMA * np.max(Q_value_batch[i])
            target = [0 for _ in range(self.action_dim)]
            target[action_batch[i]] = reward
            y_batch.append(target)
        #print("state_batch:{} y_batch: {}" .format(state_batch, y_batch))
        self.model.train_on_batch(state_batch, y_batch)

    def e_action(self, state):
        Q_value = self.model.predict(state)
        if random.random() < self.epsilon:
            return random.randint(0, self.action_dim - 1)
        else:
            return np.argmax(Q_value)
        self.epsilon -= (EPSILON - 0.01) / 10000

    def action(self, state):
        return np.argmax(self.model.predict(state))

    def load(self, model):
        self.model = load_model(model)



env = gym.make('CartPole-v0')
agent = DQN(env)
flag = 0
#agent.load('2.h5')

for i in range(10000):
    state = env.reset()
    for s in range(400):
        state = np.reshape(state, (1, agent.state_dim))
        action = agent.e_action(state)
        next_state, reward, done, _ = env.step(action)
        reward = -1 if done else 0.1
        #print("State: {}next_state:{}" .format(state, next_state))
        agent.perceive(state, action, reward, next_state, done)
        #print("State: {}" .format(state))
        state = next_state
        if done:
            break

    if i % 100 == 0:
        t_reward = 0
        for j in range(10):
            state = env.reset()
            while True:
                state = np.reshape(state, (1, agent.state_dim))
                action = agent.action(state)
                #env.render()
                state, reward, done, _ = env.step(action)
                t_reward += reward
                if done:
                    break
        print(i, t_reward / 10, len(agent.replay))
        if t_reward / 10 >= 250:
            break
    if t_reward / 10 > flag:
        flag = t_reward / 10
        save_model(agent.model, 'cartpole.h5')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 20)                100       
_________________________________________________________________
dense_5 (Dense)              (None, 20)                420       
_________________________________________________________________
dense_6 (Dense)              (None, 2)                 42        
Total params: 562.0
Trainable params: 562
Non-trainable params: 0.0
_________________________________________________________________
0 10.8 12
100 53.3 3923
200 90.5 8033
300 62.8 10000
400 46.4 10000
500 48.4 10000
600 52.7 10000
700 62.6 10000
800 58.9 10000
900 51.8 10000
1000 57.5 10000
1100 56.9 10000
1200 44.2 10000
1300 58.2 10000
1400 56.8 10000
1500 169.4 10000
1600 140.5 10000
1700 101.4 10000
1800 76.8 10000
1900 73.9 10000
2000 137.1 10000
2100 165.2 10000
2200 114.6 10000
2300 167.5 10000
2400 161.1 10000
2500 169.0 10